In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
from fexpress import Event, EventStoreSettings, ObservationDateConfig
from fexpress import FeatureExpress
from fexpress.sdk.observation_dates_config import Interval, DatePart, ObservationDatesConfigEnum
from fexpress.sdk.event_query_config import EventQueryConfigEnum

In [3]:
nba_dataset = pd.read_csv("dataset/nba/game.csv")
nba_dataset = nba_dataset[nba_dataset["game_date"] >= "2000-01-01"]
nba_dataset = nba_dataset[~nba_dataset["wl_home"].isnull()]
nba_dataset.head()

,season_id,team_id_home,team_abbreviation_home,team_name_home,game_id,game_date,matchup_home,wl_home,min,fgm_home,...,dreb_away,reb_away,ast_away,stl_away,blk_away,tov_away,pf_away,pts_away,plus_minus_away,video_available_away
35477,21999,1610612748,MIA,Miami Heat,29900423,2000-01-02 00:00:00,MIA vs. ORL,W,265,38.0,...,42.0,63.0,17.0,6.0,5.0,21.0,32.0,103,-8,0
35478,21999,1610612753,ORL,Orlando Magic,29900427,2000-01-03 00:00:00,ORL vs. DET,L,240,40.0,...,28.0,48.0,30.0,13.0,5.0,14.0,27.0,118,12,0
35479,21999,1610612764,WAS,Washington Wizards,29900426,2000-01-03 00:00:00,WAS vs. GSW,W,240,40.0,...,25.0,35.0,25.0,9.0,4.0,20.0,24.0,87,-12,0
35480,21999,1610612762,UTA,Utah Jazz,29900429,2000-01-03 00:00:00,UTA vs. DEN,W,240,44.0,...,26.0,38.0,21.0,5.0,6.0,18.0,22.0,89,-20,0
35481,21999,1610612741,CHI,Chicago Bulls,29900428,2000-01-03 00:00:00,CHI vs. POR,L,240,26.0,...,34.0,39.0,30.0,20.0,9.0,23.0,22.0,88,25,0


In [4]:
other_cols = [
    "season_id", 
    "team_id_home", 
    "team_name_home", 
    "team_name_away", 
    "game_id",
    "game_date",
    "matchup_away",
    "video_available_away",
    "video_available_home"
]
attrs = [col for col in nba_dataset.columns]
home_attr = [col for col in nba_dataset.columns if col not in other_cols and col.endswith("home")]
away_attr = [col for col in nba_dataset.columns if col not in other_cols and col.endswith("away")]

In [16]:
settings = EventStoreSettings(include_events_on_obs_date=True, parallel=True)
fx = FeatureExpress(settings)

In [17]:
for i,row in tqdm(nba_dataset.head(2000).iterrows()):
    attrs = {
            attr: row[attr] for attr in home_attr + away_attr
        }
    attrs["winning_team"] = row["team_abbreviation_home"] if row["wl_home"] == "w" else row["team_abbreviation_away"]
    event = Event(
        event_id=str(row.game_id),
        entities={"home": row["team_abbreviation_home"], "away": row["team_abbreviation_away"]},
        event_type="game",
        event_time=row["game_date"],
        attrs=attrs,
    )
    fx.new_event(event)

2000it [00:00, 5520.55it/s]


In [18]:
obs_dates_config = ObservationDatesConfigEnum.ALL_EVENTS
event_query_config = EventQueryConfigEnum.ALL_EVENTS
df = fx.query(
    obs_dates_config=obs_dates_config,
    event_query_config=event_query_config,
    query="""
    SELECT
        obs_dt as obs_dt,
        @entities.home as home,
        @entities.away as away,
        @wl_home as wl_home,
        @winning_team as winning_team,
        @team_abbreviation_home as team_home,
        lower(@team_abbreviation_home) = lower(@winning_team) as home_won,
        COUNT(*) OVER last 10 day group by wl_home as win_perc,
        MAX(event_time) over last 10 day as max_event_time,
        MIN(event_time) over last 10 day as min_event_time,
        COUNT(*) over last 10 day as count_past,
        AVG(lower(team_abbreviation_home) = lower(winning_team)) over last 10 day as home_win_perc,
        LAST(winning_team) OVER last 10 day as last_winning_team,
        FIRST(winning_team) OVER last 10 day as first_winning_team,
        LAST(event_time) OVER last 10 day WHERE team_abbreviation_home = winning_team as last_time_home_won
    FOR
        @entities := home
    """
)
df.head(100)

,obs_dt,home,away,wl_home,winning_team,team_home,home_won,win_perc,max_event_time,min_event_time,count_past,home_win_perc,last_winning_team,first_winning_team,last_time_home_won
0,2001-01-10 00:00:00,PHI,POR,L,POR,PHI,False,"{'w': 82, 'l': 56}",2001-01-10 00:00:00,2000-12-31 00:00:00,124,0.0,MIA,DET,None
1,2000-12-13 00:00:00,LAC,MIA,L,MIA,LAC,False,"{'l': 56, 'w': 104}",2000-12-13 00:00:00,2000-12-03 00:00:00,140,0.0,MIL,POR,None
2,2000-04-09 00:00:00,ORL,PHI,W,PHI,ORL,False,"{'w': 96, 'l': 64}",2000-04-09 00:00:00,2000-03-30 00:00:00,146,0.0,SAS,TOR,None
3,2000-11-03 00:00:00,ORL,PHI,L,PHI,ORL,False,"{'w': 38, 'l': 28}",2000-11-03 00:00:00,2000-10-31 00:00:00,54,0.0,TOR,CLE,None
4,2001-04-12 00:00:00,ORL,PHI,W,PHI,ORL,False,"{'l': 64, 'w': 86}",2001-04-12 00:00:00,2001-04-03 00:00:00,138,0.0,UTA,POR,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2000-01-06 00:00:00,TOR,SAC,W,SAC,TOR,False,"{'l': 22, 'w': 42}",2000-01-06 00:00:00,2000-01-02 00:00:00,50,0.0,UTA,ORL,None
96,2001-02-23 00:00:00,TOR,SAC,L,SAC,TOR,False,"{'w': 94, 'l': 74}",2001-02-23 00:00:00,2001-02-13 00:00:00,148,0.0,VAN,NYK,None
97,2000-01-05 00:00:00,ORL,VAN,W,VAN,ORL,False,"{'l': 18, 'w': 32}",2000-01-05 00:00:00,2000-01-02 00:00:00,36,0.0,CHH,ORL,None
98,2001-03-16 00:00:00,ORL,VAN,W,VAN,ORL,False,"{'w': 106, 'l': 62}",2001-03-16 00:00:00,2001-03-06 00:00:00,148,0.0,GSW,PHX,None
